In [1]:
# import matplotlib
# %matplotlib WebAgg
%matplotlib qt
import os
root = os.path.realpath(os.getcwd()+ os.sep+"..")
import sys
sys.path.append(root)
from modules.WiFi import WiFi, Modes, make_drive_command
# import pathfinder, pathfinder_visualizer
import captain
import importlib
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import time
import re, json
from datetime import datetime
import threading

Reloaded
Reloaded
Created
Reloaded
Created
Reloaded
Created
Reloaded
Created


In [2]:
# show_visual()

from ipywidgets import Button, VBox, ToggleButton, IntSlider, HBox, HTML, Layout, Output

start_pathfinder = Button(
    description="Start Captain",
    layout = Layout(width='auto', height='80px', margin='10px'), 
    style=dict(font_size="60px"),
    button_style="primary",
    disabled = captain._service.is_running())
stop_pathfinder = Button(
    description="Stop Captain", 
    layout= start_pathfinder.layout, 
    style=start_pathfinder.style,
    button_style="warning",
    disabled = not captain._service.is_running())
run_rover = ToggleButton(
    value=False, 
    description="Run Rover", 
    layout= Layout(flex='1', height='80px', margin='10px'), 
    style=dict(font_size="60px"),
    button_style="success",
    disabled = not captain._service.is_running())
visual_button = Button(
    description="PLT", 
    layout = Layout(height='auto', margin='10px'))
speed_slide = IntSlider(
    value=0, min=0, max=100, 
    orientation="vertical", 
    layout=Layout(height="auto"))
# text_display = HTML(
#     value="Some HTML text", 
#     # layout= Layout(flex='1', margin="10px", border="solid", padding="10px"))
#     layout= )
text_output = Output(layout=Layout(flex='1', max_height="500px", min_height="100px"))
poll_rover = Button(
    description="Poll Rover",
    layout = Layout(flex='1')
)
clear_output = Button(
    description="Clear Output"
)

buttons = VBox([
    start_pathfinder,
    HBox([run_rover,], layout=Layout(width='auto', padding='0px', margin='0px')),
    stop_pathfinder,
    ],
    layout = Layout(flex='1')
)
controls = HBox([
    speed_slide, buttons, speed_slide
])
logging = HBox([
    HBox([text_output], layout=Layout(border='solid', flex='1', padding='10px')), VBox([poll_rover, clear_output])
    ],
    layout = Layout(margin='10px')
)
gui = VBox([
    controls,
    logging
])

Reloaded
Created
Reloaded
Created
Starting Captain Service
6 1
6 2
6 2
6 2
6 2
6 2
6 3
6 3
6 3
6 4
6 4
6 4
6 4
6 5
6 5
6 5
6 5
6 5
6 1
6 1
6 1
6 1
6 1
6 2
6 2
6 2
6 2
6 2
6 3
6 3
6 3
6 3
6 3
6 4
6 4
6 4
6 4
6 4
6 5
6 5
6 5
6 5
6 5
6 1
6 1
6 1
6 1
6 1
6 2
6 2
6 2
23 2
23 2
21 3
21 3
21 3
21 3
21 4
21 4
21 4
21 4
21 4
21 5
21 5
21 5
21 5
21 5
21 6
21 6
21 6
21 6
21 6
21 7
21 7
21 7
21 7
21 8
21 8
21 8
21 8
21 8
21 9
21 9
21 9
21 9
21 9
21 10
21 10
2 1
2 1
2 1
2 1
4 1
4 1
4 1
4 1
4 2
4 2
4 2
4 2
4 2
4 3
4 3
3 1
3 1
4 2
4 2
4 2
3 2
3 2
3 1
4 1
4 1
3 1
3 1
3 2
3 2
4 2
4 2
4 2
4 3
4 3
4 3
4 3
4 3
4 1
4 1
4 1
4 1
4 1
4 2
4 2
4 2
4 2
4 2
4 3
5 3
5 3
5 3
5 3
5 4
5 4
5 4
5 4
4 1
4 1
4 1
4 2
4 2
4 2
4 2
4 2
3 1
3 1
4 1
4 1
4 1
4 2
4 2
4 2
4 2
4 2
4 3
4 3
4 3
4 3
4 3
4 1
4 1
4 1
4 1
4 1
4 2
4 2
4 2
4 2
4 2
4 3
4 3
4 3
5 3
5 3
5 4
5 4
5 4
5 4
4 1
4 2
5 2
5 2
5 2
5 2
56 3
6 3
5 3
5 4
5 4
5 4
6 4
6 4
5 1
5 1
4 1
4 1
4 1
4 2
4 2
4 2
4 2
4 2
4 3
4 3
5 3
5 3
5 3
5 4
5 4
5 4
5 4
5 4
4 1
4 1
5 1
5 1
5 1
5

In [3]:
def update_button_disables(pathfinding):
    start_pathfinder.disabled = pathfinding
    run_rover.disabled = not pathfinding
    stop_pathfinder.disabled = not pathfinding

def on_start_pathfinding(b):
    importlib.reload(captain)
    def command_printer(i):
        with(open("Command.txt", "w")) as f:
            f.write(f"{datetime.now().strftime('%H:%M:S')} => {i}")
    captain.start_captain_service(lambda: speed_slide.value, text_output)
    update_button_disables(True)
start_pathfinder.on_click(on_start_pathfinding)
def on_stop_pathfinding(b):
    captain.stop_captain_service()
    update_button_disables(False)
stop_pathfinder.on_click(on_stop_pathfinding)

def rover_order_changed(change):
    run = change['new']
    if run:
        run_rover.button_style='danger'
        run_rover.description='Stop Rover'
        captain.config['send_commands_to_rover'] = True
    else:
        captain.captain_stop()
        captain.config['send_commands_to_rover'] = False
        run_rover.button_style='success'
        run_rover.description='Run Rover'
run_rover.observe(rover_order_changed, 'value')
def on_poll_rover(b):
    with text_output:
        stat = captain.rover.get_status(1, 0)
        print("No status received") if stat is None else print({k:stat[k] for k in stat if re.match("^current.*", k)})
poll_rover.on_click(on_poll_rover)
clear_output.on_click(lambda b: text_output.clear_output())

In [4]:
gui

In [5]:
captain.show_captain_visual()

In [ ]:
!cls

Called start_worldview_service
Starting Worldview Service
Called start_lidar_service
Starting Lidar Service
trying to connect on port None
Actual Lidar Not connected: port None
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, 0]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -1]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -2]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -3]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -3]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -4]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -4]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -5]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'D', 'CMD': [0, -5]}
trying to connect on port None
Actual Lidar N

Exception in thread Thread-7 (start_connection):
Traceback (most recent call last):
  File "d:\Projects\autonomy_setup_test\unified-env\Lib\site-packages\websockets\legacy\protocol.py", line 963, in transfer_data


{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -44]}
Running
Captain acting


    message = await self.read_message()
              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Projects\autonomy_setup_test\unified-env\Lib\site-packages\websockets\legacy\protocol.py", line 1033, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Projects\autonomy_setup_test\unified-env\Lib\site-packages\websockets\legacy\protocol.py", line 1108, in read_data_frame
    frame = await self.read_frame(max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Projects\autonomy_setup_test\unified-env\Lib\site-packages\websockets\legacy\protocol.py", line 1165, in read_frame
    frame = await Frame.read(
            ^^^^^^^^^^^^^^^^^
  File "d:\Projects\autonomy_setup_test\unified-env\Lib\site-packages\websockets\legacy\framing.py", line 68, in read
    data = await reader(2)
           ^^^^^^^^^^^^^^^
  File "C:\Python311\Lib\asyncio\streams.py", line 733, in readexactly
    await self.

{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -44]}


...

Running
Captain acting


    self.run()
  File "d:\Projects\autonomy_setup_test\unified-env\Lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "C:\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "d:\Projects\autonomy_setup_test\unified_frameworks\sensor_array\client_lidar.py", line 55, in start_connection


{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -44]}


    asyncio.get_event_loop().run_until_complete(self.receive_data())
  File "C:\Python311\Lib\asyncio\base_events.py", line 653, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "d:\Projects\autonomy_setup_test\unified_frameworks\sensor_array\client_lidar.py", line 39, in receive_data
    data = await asyncio.wait_for(websocket.recv(), timeout=1)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Python311\Lib\asyncio\tasks.py", line 479, in wait_for
    return fut.result()
           ^^^^^^^^^^^^
  File "d:\Projects\autonomy_setup_test\unified-env\Lib\site-packages\websockets\legacy\protocol.py", line 568, in recv
    await self.ensure_open()
  File "d:\Projects\autonomy_setup_test\unified-env\Lib\site-packages\websockets\legacy\protocol.py", line 939, in ensure_open
    raise self.connection_closed_exc()
websockets.exceptions.ConnectionClosedError: no close frame received or sent


Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -44]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -44]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -44]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -44]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -44]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -44]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -44]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -44]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -44]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -44]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -44]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, 'DM': 'S', 'CMD': [0, -44]}
Running
Captain acting
{'HB': 0, 'IO': 1, 'WO': 0, '